# TP2 : Reconstruction tomographique
### Physique Numérique (PHY-3500)
### Par: Simon Gauthier, Laurent Émond, Thomas Charland
### Présenté à: Xavier Roy-Pomerleau
### Remis le: 23 mars 2025

## Mise en contexte

L'arrivée de l'ordinateur a révolutionné la médecine moderne en permettant de développer des méthodes nécessitant sur une puissance de calcul élevée. La tomodensitométrie (TDM), qui permet d'estimer la distribution de la densité d'un objet à partir de plusieurs mesures d'atténuation [1], est l'une de ces méthodes. La tomodensitométrie est majoritairement tulisée pour l'analyse de zones non-visibles par d'autresd examens d'imagerie médicale [2], et permet d'obtenir des résultat de l'ordre du millimètre [1]

Ce document résout le TP2 du cours de Physique Numérique (PHY-3500). 

In [3]:
#Importer toutes les libraries nécessaires pour la résolution du TP
import numpy as np
import matplotlib.pyplot as plt
import math

# Numéro 1

En utilisant la base de donnée XCOM du NIST [3], nous obtenons un coefficient d'atténuation $\mu = 2.059 \cdot 10^{-1}$ cm^2/g pour un photon de 60KeV voyageant dans l'eau à 1.0 g/cm^3. 

In [ ]:
#Code pour résoudre le numéro 1
def remaining_intensity(attenuation_coefs_of_media : list, distance_travelled_in_media : list):
    sum_value = 0
    for i in range(len(attenuation_coefs_of_media)):
        sum_value += attenuation_coefs_of_media[i] * distance_travelled_in_media[i]
    
    remaining_intensity = 100 * math.e**(-sum_value)
    return remaining_intensity / 100

print("La fraction du signal subsistant après 5cm est: ", remaining_intensity([0.2059], [5]))
print("La fraction du signal subsistant après 20cm est: ", remaining_intensity([0.2059], [20]))

La fraction du signal subsistant après 5cm est:  0.35718550868273846
La fraction du signal subsistant après 20cm est:  0.016277035981531945


Après avoir traversé 5cm d'eau, la fraction du signal subsistante est de 35.72%. Après avoir traversé 20cm d'eau, la fraction du signal subsistante est de 1.62%. 

# Numéro 2

In [ ]:
#Code pour résoudre le numéro 2


# Numéro 3

In [ ]:
#Code pour résoudre le numéro 3

# Numéro 4

In [ ]:
#Code pour résoudre le numéro 4

# Numéro 5

In [ ]:
#Code pour résoudre le numéro 5

# Bibliographie

[1] TP2 : Reconstruction tomographique, D. Matenine et P. Després, *PHY-3500– Physique num´erique (H25)*

[2] Pourquoi réaliser une tomodentometrie, Information hospitalière, <https://www.informationhospitaliere.com/tomodensitometrie-deroulement-precautions-et-effets-secondaires#Pourquoi_realiser_une_tomodensitometrie%E2%80%89>

[3] XCOM database, NIST, <https://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html>

[4]